In [140]:
import random
import networkx as nx
import itertools
import time

In [141]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [142]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [143]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [144]:
def fitness(resolving_set,G,shortest_paths,all_pairs):
    unresolved_pairs = 0
    for u, v in all_pairs:
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [145]:
def construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths,all_pairs):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:
        # Računanje verovatnoće izbora svakog čvora
        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}
        total = sum(probabilities.values())
        
        if total == 0:
            break
        
        # Normalizacija verovatnoća
        probabilities = {node: prob / total for node, prob in probabilities.items()}

        # Izbor čvora proporcionalno verovatnoći
        selected_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        # Ako je rešenje dovoljno dobro, prekidamo ranije
        if fitness(resolving_set,G,shortest_paths,all_pairs) < 100:
            break

    return resolving_set

In [146]:
def update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths,all_pairs):
    # Isparavanje feromona
    for node in pheromones:
        pheromones[node] *= (1 - evaporation)
    
    # Dodavanje novih feromona na čvorove iz najboljeg rešenja
    for node in best_solution:
        pheromones[node] += Q / fitness(best_solution,G,shortest_paths,all_pairs)

In [147]:
def ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths,all_pairs):
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(num_iterations):
        solutions = [construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths,all_pairs) for _ in range(num_ants)]
        sorted_solutions = sorted(solutions, key=lambda s: fitness(s, G,shortest_paths,all_pairs))        
        if fitness(sorted_solutions[0],G,shortest_paths,all_pairs) < best_fitness:
            best_solution = sorted_solutions[0]
            best_fitness = fitness(best_solution,G,shortest_paths,all_pairs)

        update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths,all_pairs)
        
        print(f"Iteracija {iteration}, najbolji skup {best_fitness},  {best_solution}")
    print("\nOptimalni rešavajući skup:", best_solution, 'velicine:',len(best_solution))
    return best_solution

In [148]:
G = load_graph('grafovi\mali grafovi\hamming6-2.txt')
n = len(G.nodes)

num_ants = 50    
num_iterations = 10 
alpha = 2.0
beta = 1.0
evaporation = 0.1
Q = 100  

# Inicijalizacija feromona
pheromones = {node: 1.0 for node in G.nodes}

# heuristics = {node: 1.0 / (sum(nx.shortest_path_length(G, node, j) for j in G.nodes if j != node) + 1) for node in G.nodes}
start_time1 = time.time()

centrality = nx.betweenness_centrality(G)
heuristics = {node: centrality[node] for node in G.nodes}

shortest_paths = dict(nx.all_pairs_shortest_path_length(G))  # Svi putevi između svih parova
all_pairs = list(itertools.combinations(G.nodes, 2))

rez = ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths,all_pairs)
end_time1 = time.time()
print("Vrijeme ", end_time1-start_time1)


<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_374568\2988317559.py:1: SyntaxWarning: invalid escape sequence '\m'
  G = load_graph('grafovi\mali grafovi\hamming6-2.txt')


Iteracija 0, najbolji skup 24,  {2, 11, 13, 18, 21, 22, 23, 25, 27, 28, 31, 32, 35, 36, 38, 39, 41, 42, 46, 50, 55, 61, 62, 63}
Iteracija 1, najbolji skup 22,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 42, 50, 55, 61, 62, 63}
Iteracija 2, najbolji skup 22,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 42, 50, 55, 61, 62, 63}
Iteracija 3, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 50, 55, 61, 62, 63}
Iteracija 4, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 50, 55, 61, 62, 63}
Iteracija 5, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 50, 55, 61, 62, 63}
Iteracija 6, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 50, 55, 61, 62, 63}
Iteracija 7, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 23, 25, 28, 31, 35, 36, 38, 39, 41, 50, 55, 61, 62, 63}
Iteracija 8, najbolji skup 21,  {2, 8, 10, 11, 13, 18, 22, 

In [149]:
is_resolving_set(rez,G)

True